# Predictions

-   Load current model
-   Tabulate simple rule based predictions (random, home, spread, and elo)
-   Graph the outputs

In [57]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

# Add the project root to the Python path
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.dirname(notebook_dir)
sys.path.append(project_root)

from src.utils.team_pairs import TEAM_PAIRS

## Load Model Predictions

In [86]:
# Load predictions data
predictions_df = pd.read_parquet('../models/win_probability/predictions_2024_3.parquet')

# Rename win_probability column to model_1
predictions_df = predictions_df.rename(columns={'win_probability': 'model_1'})

# Display first few rows and basic info
print("Dataset Shape:", predictions_df.shape)
predictions_df.head()

Dataset Shape: (532, 7)


,year,week,home_id,home_team,away_id,away_team,model_1
0,2024,3,2633,Tennessee,2309,Kent State,0.961533
1,2024,4,251,Texas,2433,Louisiana Monroe,0.960478
2,2024,13,61,Georgia,113,UMass,0.956873
3,2024,4,213,Penn State,2309,Kent State,0.956404
4,2024,13,2633,Tennessee,2638,UTEP,0.955330


## Load All Features

In [59]:
target_db_path = '../data/04_features/features_teams.db'
conn = sqlite3.connect(target_db_path)

# Read the data into a DataFrame
df_all_years = pd.read_sql_query("SELECT * FROM features_teams", conn)

# Close the connection
conn.close()

df_all_years.head()

,year,week,start_date,is_home,season_type,neutral_site,conference_game,team_id,opponent_id,team_conference,...,defense_standard_downs_success_rate_last_3,defense_standard_downs_success_rate_last_10,win_rate_last_1,win_rate_last_3,win_rate_last_5,win_rate_last_10,points_scored_last_1,points_allowed_last_1,points_scored_last_3,points_allowed_last_3
0,2001,2,2001-08-30T23:30:00.000Z,1,regular,0,0,8,2439,SEC,...,0.528311,0.530178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2001,2,2001-08-31T00:00:00.000Z,1,regular,0,0,238,2393,SEC,...,0.568602,0.562389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2001,2,2001-09-01T16:30:00.000Z,1,regular,0,0,96,97,SEC,...,0.598326,0.579564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2001,2,2001-09-01T17:00:00.000Z,1,regular,0,0,61,2032,SEC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2001,2,2001-09-01T20:00:00.000Z,1,regular,0,0,2633,183,SEC,...,0.558417,0.572578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Load Pick Em Outputs

In [83]:
# Load and flatten matchups data
import json
from itertools import chain

with open('../data/00_temp/matchups.json', 'r') as f:
    matchups = json.load(f)

# Flatten the nested structure
flattened_games = []
for week, data in matchups.items():
    week_num = int(week.replace('week', ''))
    for game in data['games']:
        flattened_games.append({
            'week': week_num,
            'team_a': game['team_a'],
            'team_a_odds': game['team_a_odds'],
            'team_a_pick_percentage': game['team_a_pick_percentage'],
            'team_b': game['team_b'],
            'team_b_odds': game['team_b_odds'],
            'team_b_pick_percentage': game['team_b_pick_percentage']
        })

matchups_df = pd.DataFrame(flattened_games)

# Sort by week and display first few rows
matchups_df = matchups_df.sort_values('week')
matchups_df.head()

,week,team_a,team_a_odds,team_a_pick_percentage,team_b,team_b_odds,team_b_pick_percentage
0,1,Miami,-125,50,Florida,105,50
1,1,Notre Dame,130,53,Texas A&M,-150,47
2,1,Miami (OH),130,9,Northwestern,-150,91
3,1,North Texas,170,12,South Alabama,-200,88
4,1,Wyoming,200,6,Arizona State,-240,94


## Load Raw Data as Needed

In [61]:
# Connect to database
conn = sqlite3.connect('../data/02_interim/college_football.db')

# Query 2024 games and unstack data column
query = """
SELECT *
FROM games 
WHERE season = 2024
"""

# Load 2024 games data
conn = sqlite3.connect('../data/02_interim/college_football.db')
games_2024_df = pd.read_sql_query(query, conn)
conn.close()

games_2024_df.head()

,id,season,week,season_type,start_date,start_time_tbd,completed,neutral_site,conference_game,attendance,...,away_conference,away_division,away_points,away_line_scores,away_post_win_prob,away_pregame_elo,away_postgame_elo,excitement_index,highlights,notes
0,401628320,2024,1,regular,2024-08-29T23:30:00.000Z,0,1,1,0,None,...,SWAC,fcs,0.0,"[0, 0, 0, 0]",0.001030,NaN,NaN,2.573553,None,None
1,401628327,2024,1,regular,2024-08-30T00:00:00.000Z,0,1,0,0,None,...,MVFC,fcs,0.0,"[0, 0, 0, 0]",0.001092,NaN,NaN,2.195767,None,None
2,401628328,2024,1,regular,2024-08-30T23:00:00.000Z,0,1,0,0,None,...,American Athletic,fbs,3.0,"[0, 0, 3, 0]",0.003269,1178.0,1115.0,2.672526,None,None
3,401628323,2024,1,regular,2024-08-31T16:00:00.000Z,0,1,1,0,None,...,ACC,fbs,3.0,"[0, 0, 3, 0]",0.003202,1676.0,1625.0,3.998181,None,Aflac Kickoff
4,401628333,2024,1,regular,2024-08-31T16:00:00.000Z,0,1,0,0,None,...,ACC,fbs,27.0,"[0, 3, 10, 14, 0]",0.024029,1604.0,1575.0,7.779942,None,None


## Start Building Main Prediction Comparison DataFrame

In [137]:
# Create a copy of matchups_df
df = matchups_df.copy()

# Create a dictionary for quick team name to ID lookup
team_dict = {name: id for id, name in TEAM_PAIRS}

# Dictionary for name standardization
name_standardization = {
    'App State': 'Appalachian State',
    'UL Monroe': 'Louisiana Monroe',
    'Sam Houston': 'Sam Houston State'
}

# Function to standardize team names
def standardize_team_name(team_name):
    return name_standardization.get(team_name, team_name)

# Function to find closest match for team names
def get_team_id(team_name):
    # Standardize name first
    team_name = standardize_team_name(team_name)
    
    # Direct lookup
    if team_name in team_dict:
        return team_dict[team_name]
    
    # Handle common variations
    cleaned_name = team_name.replace('(', '').replace(')', '').strip()
    if cleaned_name in team_dict:
        return team_dict[cleaned_name]
    
    # Could add more fuzzy matching here if needed
    return None

# Add ID columns
df['team_a_id'] = df['team_a'].apply(get_team_id)
df['team_b_id'] = df['team_b'].apply(get_team_id)

# Display results to check matching
print("Matching results:")
print(f"Total rows: {len(df)}")
print(f"Rows with matched team_a_id: {df['team_a_id'].notna().sum()}")
print(f"Rows with matched team_b_id: {df['team_b_id'].notna().sum()}")

# Show any unmatched teams
unmatched_a = df[df['team_a_id'].isna()]['team_a'].unique()
unmatched_b = df[df['team_b_id'].isna()]['team_b'].unique()
if len(unmatched_a) > 0 or len(unmatched_b) > 0:
    print("\nUnmatched teams:")
    print("Team A:", unmatched_a)
    print("Team B:", unmatched_b)

Matching results:
Total rows: 140
Rows with matched team_a_id: 140
Rows with matched team_b_id: 140


In [138]:
# Create single merge with team_b as home and keep only needed columns
df = df.merge(
    predictions_df[['week', 'home_id', 'away_id', 'model_1']],
    left_on=['week', 'team_b_id', 'team_a_id'],  # team_b is home, team_a is away
    right_on=['week', 'home_id', 'away_id'],
    how='left'
)

# Flip the probability and drop unnecessary columns
df['model_1'] = 1 - df['model_1']
df = df.drop(['home_id', 'away_id'], axis=1)

# Fill null values with 0.5
df['model_1'] = df['model_1'].fillna(0.5)

# Print matching diagnostics
print("Matching results:")
print(f"Total games in original df: {len(df)}")
print(f"Games with predictions: {df['model_1'].notna().sum()}")
print(f"Games missing predictions (filled with 0.5): {(df['model_1'] == 0.5).sum()}")

# Show games that were filled with 0.5
filled_games = df[df['model_1'] == 0.5][['week', 'team_a', 'team_b', 'team_a_id', 'team_b_id']]
if not filled_games.empty:
    print("\nGames filled with 0.5 probability:")
    display(filled_games)

Matching results:
Total games in original df: 140
Games with predictions: 140
Games missing predictions (filled with 0.5): 23

Games filled with 0.5 probability:


,week,team_a,team_b,team_a_id,team_b_id
3,1,North Texas,South Alabama,249,6
7,1,Boise State,Georgia Southern,68,290
28,3,App State,East Carolina,2026,151
31,4,Toledo,Western Kentucky,2649,98
44,5,Liberty,App State,2335,2026
49,5,Fresno State,UNLV,278,2439
72,8,Louisiana,Coastal Carolina,309,324
85,9,UL Monroe,South Alabama,2433,6
88,9,Bowling Green,Toledo,189,2649
90,10,Colorado State,Nevada,36,2440


In [139]:
df.head()

,week,team_a,team_a_odds,team_a_pick_percentage,team_b,team_b_odds,team_b_pick_percentage,team_a_id,team_b_id,model_1
0,1,Miami,-125,50,Florida,105,50,2390,57,0.461737
1,1,Notre Dame,130,53,Texas A&M,-150,47,87,245,0.631888
2,1,Miami (OH),130,9,Northwestern,-150,91,193,77,0.332406
3,1,North Texas,170,12,South Alabama,-200,88,249,6,0.500000
4,1,Wyoming,200,6,Arizona State,-240,94,2751,9,0.315414
